In [1]:
from tensorflow.keras.layers import Dropout, Dense, GRU, Embedding,LSTM
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import model_selection, naive_bayes, svm

In [2]:
from tensorflow.keras.layers import Dropout, Dense,Input,Embedding,Flatten, MaxPooling1D, Conv1D
from tensorflow.keras.models import Sequential,Model
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.layers import Concatenate

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path="/content/gdrive/MyDrive/Github/Detection_Violence/DL"

In [5]:
Vector_path="/content/gdrive/MyDrive/Github/LDA/"

In [6]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open(Vector_path+"glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [7]:
import nltk 
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


# Violence Classification based CNN algorithm

In [8]:
import tensorflow as tf

def Build_Model_CNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):

    """
        def buildModel_CNN(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
        word_index in word index ,
        embeddings_index is embeddings index, look at data_helper.py
        nClasses is number of classes,
        MAX_SEQUENCE_LENGTH is maximum lenght of text sequences,
        EMBEDDING_DIM is an int value for dimention of word embedding look at data_helper.py
    """

    model = Sequential()
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)

    # applying a more complex convolutional approach
    convs = []
    filter_sizes = []
    layer = 5
    print("Filter  ",layer)
    for fl in range(0,layer):
        filter_sizes.append((fl+2))

    node = 128
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    for fsz in filter_sizes:
        l_conv = Conv1D(node, kernel_size=fsz, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(5)(l_conv)
        #l_pool = Dropout(0.25)(l_pool)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)
    l_cov1 = Conv1D(node, 5, activation='relu')(l_merge)
    l_cov1 = Dropout(dropout)(l_cov1)
    l_pool1 = MaxPooling1D(5)(l_cov1)
    l_cov2 = Conv1D(node, 5, activation='relu')(l_pool1)
    l_cov2 = Dropout(dropout)(l_cov2)
    l_pool2 = MaxPooling1D(30)(l_cov2)
    l_flat = Flatten()(l_pool2)
    l_dense = Dense(1024, activation='relu')(l_flat)
    l_dense = Dropout(dropout)(l_dense)
    l_dense = Dense(128, activation='relu')(l_dense)
    l_dense = Dropout(dropout)(l_dense)
    preds = Dense(nclasses, activation='softmax')(l_dense)
    model = Model(sequence_input, preds)
    
    learning_rate = 0.001
    epochs=2
    opt= tf.keras.optimizers.Adam(learning_rate=learning_rate , decay=learning_rate/(epochs*0.5))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])



    return model

In [9]:
Corpus = pd.read_csv(Vector_path+"Violence in Yemens Social Media.csv", sep=';')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
import nltk 
words = set(nltk.corpus.words.words())
Corpus = pd.read_csv(Vector_path+"Violence in Yemens Social Media.csv", sep=';')
list_term=[]
for term in Corpus.Text:
    sent = " ".join(w for w in nltk.wordpunct_tokenize(term) if w.lower() in words or not w.isalpha())
    list_term.append(sent)
dic={'Text':list_term, 'Dominant_Topic':Corpus.Dominant_Topic}
Corpus=pd.DataFrame(dic)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:

X_train, X_test, y_train, y_test = model_selection.train_test_split(Corpus['Text'],Corpus.Dominant_Topic,test_size=0.2)




In [12]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)



Found 7163 unique tokens.
(77062, 500)
Total 400000 word vectors.


In [13]:



model_CNN = Build_Model_CNN_Text(word_index,embeddings_index, 5)

model_CNN.summary()

model_CNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=15,
                              batch_size=128,
                              verbose=2)



Filter   5
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 50)      358200      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 499, 128)     12928       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 498, 128)     19328       ['embedding[0][0]']              
                                                                                   

In [14]:
Topic0="Religious organization violence"
Topic1="Houthi militias terrorize Yemeni people"
Topic3="international peace threats"
Topic4="Racist Tendencies"
Topic5="Recruitment and killing of children"

In [15]:
predicted = model_CNN.predict(X_test_Glove)
predicted = np.argmax(predicted, axis=1)
from sklearn.metrics import classification_report
target_names = [Topic0, Topic1, Topic3, Topic4, Topic5]

In [16]:
report=classification_report(y_test, predicted, target_names=target_names, digits=4)
print(report)

                                         precision    recall  f1-score   support

        Religious organization violence     0.9852    0.9627    0.9738      1314
Houthi militias terrorize Yemeni people     0.9818    0.9682    0.9750      8524
            international peace threats     0.9672    0.9502    0.9587      1025
                      Racist Tendencies     0.9586    0.9664    0.9625      1965
    Recruitment and killing of children     0.9060    0.9586    0.9316      2585

                               accuracy                         0.9647     15413
                              macro avg     0.9598    0.9612    0.9603     15413
                           weighted avg     0.9655    0.9647    0.9649     15413

